In [ ]:
import re

class preProcess:

    def __init__(self):
        
            self.string = ""
            self.function_string = ""
            self.function_string_temp = ""
            self.all_string = ""
            self.all_variable =[]
            self.all_variable_name = []
            self.struct_type_array = []
            self.struct_variable = []

            self.regex_array = [" int (.*?);"," float (.*?);"," char (.*?);"," boolean (.*?);"," double (.*?);"," short (.*?);", " int8_t (.*?);",
                     " uint8_t (.*?);"," int16_t (.*?);", " uint16_t (.*?);", " int32_t (.*?);", " uint32_t (.*?);",
                     " int64_t (.*?);"," uint64_t (.*?);",
                      " short int (.*?);"," unsigned int (.*?);",
                      " long int (.*?);"," long long int (.*?);",
                      " unsigned long int (.*?);"," unsigned long long int (.*?);",
                      " signed char (.*?);"," unsigned char (.*?);",
                      " long double (.*?);",
                      " _Bool (.*?);", " wchar_t(.*?);"
                     ]

            self.regex_provision = ["intMarker","floatMarker","charMarker","booleanMarker","doubleMarker","shortMarker", "int8_tMarker",
                         "uint8_tMarker","int16_tMarker","uint16_tMarker",
                          "int32_tMarker",
                         "uint32_tMarker",
                          "int64_tMarker",
                         "uint64_tMarker",
                          "shortintMarker","unsignedintMarker",
                          "longintMarker","longlongintMarker",
                          "unsignedlongintMarker","unsignedlonglongintMarker",
                          "signedcharMarker","unsignedcharMarker",
                          "longdoubleMarker","_BoolMarker","wchar_tMarker"
                         ]

            self.regex_function_name = [" int(.*?)\(",  " void(.*?)\("," float(.*?)\("," Double(.*?)\("," char(.*?)\(",
                            " int8_t(.*?)\("," uint8_t(.*?)\("," int16_t(.*?)\("," uint16_t(.*?)\("," int32_t(.*?)\("," uint32_t(.*?)\(",
                             " int64_t(.*?)\("," uint64_t(.*?)\(", " _Bool(.*?)\("
                            ]

            self.regex_function_name_provision = ["IntFunctionMarker",
                          "VoidFunctionMarker","FloatFunctionMarker","DoubleFunctionMarker","CharFunctionMarker",
                          "Signed8BitsIntFunctionMarker","Unsigned8BitsIntFunctionMarker","Signed16BitsIntFunctionMarker",
                          "Unsigned16BitsIntFunctionMarker","Signed32BitsIntFunctionMarker",
                          "Unsigned32BitsIntFunctionMarker","Signed64BitsIntFunctionMarker"
                                      ,"Unsigned64BitsIntFunctionMarker",  
                          "BooleanFunctionMarker"
                                     ]

            self.regex_pointer = ["(.*?)\*(.*?)[,\()=;]","(.*?)\* (.*?)[,\()=;]"]
            self.regex_define = ["#define(.*?)\n"]
            self.regex_define_provision = ["defineMarker"]
            self.regex_include = ["#include(.*?)<(.*?)>","#include(.*?)\"(.*?)\""]
            self.regex_include_provision = ["includeMarker","includeMarker"]
            self.regex_struct = [r"\bstruct\b(.*?){",r"\bstruct\b(.*?);"]
            self.regex_struct_provision = [" StructMarker","StructMarker"]
            self.marker_array = ["MarkerInt","MarkerFloat","MarkerChar","MarkerBoolean","MarkerDouble","MarkerShort",
                      "IntFunctionMarker",
                          "VoidFunctionMarker","FloatFunctionMarker","DoubleFunctionMarker","CharFunctionMarker",
                       "defineMarker", "includeMarker", "StructMarker","StructMarker"
                      ]

    def start(self,txt):
        self.string = txt
        self.delete_comment_line()
        self.function_name_marker()
        self.funciton_paramater() 
        self.string = self.string + self.function_string_temp
        self.string = self.execute_all(self.string)
        return self.string

    def delete_comment_line(self):
        start = self.string.find("/*")
        end = self.string.find("*/")
        include_number = self.string.count('/*')
        index = 0
        
        while index < include_number:
            start = self.string.find("/*")
            end = self.string.find("*/")
            self.string = self.string.replace(self.string[start:end+2],"")
            index = index + 1

        self.string = re.sub("\/\/(.*?)\n", "", self.string)
        self.string = self.string.replace("\t"," ")
        
    def function_name_marker(self):
        regex_array_length = len(self.regex_function_name)
        regex_index = 0
        counter = 0
        while counter < regex_array_length:
            iter = re.finditer(self.regex_function_name[counter],self.string)
            for m in iter:
                self.replace_function_name(m.group(1),self.regex_function_name_provision[counter])
            counter = counter+1
    
    def add_variable(self, element, m, marker):
        new_variable = Degisken(m.start(), m.end(), element, marker)
        self.all_variable.append(new_variable)
        self.all_variable_name.append(element)
        
    def replace_function(self, current_string, regex_response_list, marker, string):
        if (type(current_string) == tuple):
            temp = current_string[1];
            if "*" in temp:

                return string;
            self.add_variable(temp,regex_response_list,marker)
        else :
            current_string = current_string.replace(" ","")
            string_check= re.compile('[@!#$%^&<>?\/|}{~:]')

            if "*" in current_string:

                return string;

            if(string_check.search(current_string) == None):
                temp = current_string.replace(" ","")
                if ("[" and "]" in temp and ")" not in temp):
                    index = temp.find("[")
                    temp = temp[0:index]
                    new_marker = marker + "Array"
                    self.add_variable(temp, regex_response_list, new_marker)

                elif (")" not in temp):
                    self.add_variable(temp, regex_response_list, marker) 
            else: 
                temp = re.sub(r'[^\w]',"",current_string)
                temp = temp.replace(" ","")
                self.add_variable(temp,regex_response_list,marker)
    
    def replace_function_name(self, function_name, key):
        function_name = function_name.strip();

        if "{" in function_name:
            return

        if "}" in function_name:
            return

        if "[" in function_name:
            return

        if "]" in function_name:
            return

        if ")" in function_name:
            return

        if "+" in function_name:
            return

        if "?" in function_name:
            return

        if "=" in function_name:
            return

        if ":" in function_name:
            return

        if "<" in function_name:
            return

        if ">" in function_name:
            return

        if "#" in function_name:
            return

        if "&" in function_name:
            return

        if "%" in function_name:
            return

        if "~" in function_name:
            return

        if "!" in function_name:
            return

        if "\\" in function_name:
            return

        if "/" in function_name:
            return

        if "|" in function_name:
            return

        if "*" in function_name:
            function_name = function_name.replace("*","")
            key = "Pointer" + key
            temp = r'\b'+function_name+'\\b'
            pattern = re.compile(temp)
            self.string = re.sub(pattern=pattern, string=self.string, repl=key)
        else:
            temp = r'\b'+function_name+'\\b'
            pattern = re.compile(temp)
            self.string = re.sub(pattern=pattern, string=self.string, repl=key)
    
    def replace_func(self,array_dict):
        for key, value in array_dict.items():
            string_check= re.compile('[@!#$%^&()<>?\/|[\]}{~:]')

            if "*" in value:
                continue;

            if string_check.search(value) != None:
                temp = re.sub(r'[^\w]'," ",value)
                temp = temp.replace(" ","")
                temp = r'\b'+temp+'\\b'
                pattern = re.compile(temp)
                replaceValue = key+"Marker"
                self.function_string = re.sub(pattern=pattern, string=self.function_string, repl=replaceValue)
            else:
                temp = r'\b'+value+'\\b'
                pattern = re.compile(temp)
                replaceValue = key+"Marker"
                self.function_string = re.sub(pattern=pattern, string=self.function_string, repl=replaceValue)
     
    def funciton_paramater(self): 
        line_index = 0
        string_temp = ""
        string_temp = self.string
        function_start =0
        function_end = 0
        start_temp = 5
        self.function_string_temp = ""
        array_dict = dict()
        start_index = 0
        find_index = 0
        index = 0

        for item in re.finditer('(.*?)FunctionMarker(.*?)', string_temp):
            new_string = string_temp[item.start():] 
            if ";" in new_string.splitlines()[0]:
                continue;
            if "FunctionMarker()" in new_string.splitlines()[0]:
                  for line in new_string.splitlines():
                    self.function_string += line + "\n"
                    line_index = line_index + 1
                    if "{" in line:
                        function_start = function_start + 1
                        start_temp = function_start
                    if "}" in line:
                        function_end = function_end + 1
                    if (start_temp == function_end and start_temp != 0):
                        temp=start_index
                        if self.function_string in self.string[start_index:]:
                            find_index = self.string[start_index:].index(self.function_string)
                        start_index = find_index
                        len_index = len(self.function_string)
                        end_index = start_index + len_index
                        self.string = self.string.replace(self.string[start_index:end_index],"")
                        break;
                        
            else:
                for line in new_string.splitlines():
                    self.function_string += line + "\n"
                    
                    line_index = line_index + 1
                    if "{" in line:
                        function_start = function_start + 1
                        start_temp = function_start
                    if "}" in line:
                        function_end = function_end + 1
                    if (start_temp == function_end and start_temp != 0):
                        temp=start_index
                        if self.function_string in self.string[start_index:]:
                            find_index = self.string[start_index:].index(self.function_string)
                        start_index = find_index
                        len_index = len(self.function_string)
                        end_index = start_index + len_index
                        self.string = self.string.replace(self.string[start_index:end_index],"")
                        parameters = re.findall("FunctionMarker\s*\((.*?)\)",self.function_string)
                        functionDict = dict()
                        for elem in parameters:
                                if elem == "":
                                    continue;
                                virgulSplit = elem.split(",")
                                for element in virgulSplit:
                                    if element[0] == " ":
                                        element = element[1:]
                                    if element[len(element)-1] == " ":
                                        element = element[:-1]
                                    element = element.split(" ")
                                    if len(element) == 2:
                                        array_dict[element[0]] = element[1]
                                        self.replace_func(array_dict)
                                    elif len(element) > 2:
                                        array_dict[element[0]] = element[len(element)-1]
                                        self.replace_func(array_dict)
                        break;
            temp_function_string = self.execute_all(self.function_string)
            temp_function_string = temp_function_string + "\n"
            self.function_string_temp = self.function_string_temp + temp_function_string
            self.function_string = ""
            end_index = 0
            start_index=0
            start_temp = 0

    def execute_all(self,new_string):
        new_string = self.marker_variable_and_include(new_string)
        new_string = self.struct_marker(new_string)
        new_string = self.find_pointer(new_string)
        new_string = self.marker_define(new_string)
        self.all_variable =[]
        self.all_variable_name =[]
        self.struct_type_array =  []
        self.struct_variable = []

        return new_string; 
    
    def marker_variable_and_include(self,string):
        regex_array_length = len(self.regex_array)
        regex_index = 0
        counter = 0
        while counter < regex_array_length:
            iter = re.finditer(self.regex_array[counter],string)
            for m in iter:
                self.divide(m,self.regex_provision[counter],string)
            counter = counter+1

        string = self.replace_for_all(string)

        regex_include_length = len(self.regex_include)
        counter = 0
        while counter < regex_include_length:
            m = re.findall(self.regex_include[counter],string)
            if (len(m) > 0):
                string = self.divide_include(m,string)  
            counter = counter+1

        return string
        
        
    def divide(self,regex_response_list,marker,string):
        element = regex_response_list.group(1)
        if "," in element and "=" in element:
            comma_array = element.split(",")
            for splited in comma_array:
                temp = splited.split("=")
                self.replace_function(temp[0], regex_response_list, marker, string)  


        elif "," in element:
            comma_array = element.split(",")
            self.replace_func_array(comma_array, regex_response_list, marker, string)


        elif "=" in element:
            equal_array = element.split("=")
            self.replace_function(equal_array[0], regex_response_list, marker, string)           
        else:
            self.replace_function(element, regex_response_list, marker, string)
            
    
    def replace_func_array(self, string_array, regex_response_list ,marker, string): 
        for element in string_array:
            if "*" in element:
                continue
            element = element.replace(" ","")
            self.add_variable(element, regex_response_list, marker)
            
    
    def replace_for_all(self, string):
        for variable in self.all_variable:
            if variable.variable_name in self.marker_array:
                continue;

            if "marker" in variable.variable_name or "Marker" in variable.variable_name:
              continue

            else:    
                try:
                  if self.all_variable_name.count(variable.variable_name) > 1:
                    string = self.change_for_same(variable,string)
                  else: 
                    temp = r'\b'+variable.variable_name+'\\b'
                    pattern = re.compile(temp)
                    start = variable.start_indexx;
                    tempString = string[start:]
                    self.change_index(variable,string,0)
                    string = string.replace(string[start:],"")
                    tempString = re.sub(pattern=pattern, string=tempString, repl=variable.variable_key)
                    string = string[:start] + tempString
                except Exception as e:
                  continue
        return string
    
    
    def change_for_same(self, variable, string):
        min = variable.start_index;
        max = variable.start_index;
        same_variable = []
        indices = [i for i, x in enumerate(self.all_variable_name) if x == variable.variable_name]

        for indice in indices:
            same_variable.append(self.all_variable[indice].start_index)

        same_variable.sort()

        find_index = same_variable.index(variable.start_index)

        temp = r'\b'+variable.variable_name+'\\b'
        pattern = re.compile(temp)

        start = same_variable[find_index]

        if find_index < len(same_variable)-1:
            end = same_variable[find_index+1]
            temp = string[start:end]
            self.change_index(variable,string,end)
            string = string.replace(string[start:end],"")
            temp = re.sub(pattern=pattern, string=temp,repl=variable.variable_key)
            string = string[:start] + temp + string[start:]

        else :
            temp = string[start:]
            self.change_index(variable,string,0)
            string = string.replace(string[start:],"")
            temp = re.sub(pattern=pattern, string=temp,repl=variable.variable_key)
            string = string[:start] + temp + string[start:]

        return string
    
    
    def change_index(self, variable, string, end):

        if variable.variable_name == variable.variable_key:
            return;

        upper_index = [i for i, x in enumerate(self.all_variable) if x.start_index > variable.start_index]
        temp = r'\b'+variable.variable_name+'\\b'
        pattern = re.compile(temp)
        length_diff = len(variable.variable_key) - len(variable.variable_name)

        for i in upper_index:

            if self.all_variable[i].variable_name in self.marker_array:
                continue;

            else:
                if end == 0:
                    sub_matches = re.findall(pattern, string[variable.start_index:self.all_variable[i].start_index]) 
                    len_sub_matches = len(sub_matches)
                    self.all_variable[i].start_index = self.all_variable[i].start_index + (len_sub_matches * length_diff)

                else:
                    if(self.all_variable[i].start_index > end):
                        matches = re.findall(pattern,string[variable.start_index:end])
                        sub_matches = len(matches)
                        self.all_variable[i].start_index = self.all_variable[i].start_index + (sub_matches * length_diff)
                    else:
                        sub_matches = re.findall(pattern, string[variable.start_index:self.all_variable[i].start_index]) 
                        len_sub_matches = len(sub_matches)
                        self.all_variable[i].start_index = self.all_variable[i].start_index + (len_sub_matches * length_diff)


    def change_define(self, defined_strings, string):
        splited_defined_dict = dict()
        for value in defined_strings:
            splited_defined = value.split(" ")
            if len(splited_defined)<=2:
                continue;
            else:
                splited_defined_dict[splited_defined[1]] = splited_defined[2]

        for key, value in splited_defined_dict.items():
            temp = r'\b'+key+'\\b'
            pattern = re.compile(temp)
            string = re.sub(pattern=pattern, string=string, repl="DefineMarker")
            
        return string
    
    
    def divide_include(self,current_string,string):
        for elem in current_string:
            if (type(elem) == tuple):
                temp = elem[1]
                temp = r'\b'+temp+'\\b'
                pattern = re.compile(temp)
                string = re.sub(pattern=pattern, string=string, repl="IncludeMarker")
            else:
                continue;

        return string
    
    
    def struct_type_mark(self, struct_type_array, provision, string):
        for elem in struct_type_array:
            if elem = ="":
                continue
            temp = r'\b'+elem+'\\b'
            pattern = re.compile(temp)
            string = re.sub(pattern=pattern, string=string, repl=provision)
        return string
    
    
    def struct_variable_mark(self, struct_variable_array, provision, string):
        for elem in struct_variable_array:
            if elem == "":
                continue
            temp = r'\b'+elem+'\\b'
            pattern = re.compile(temp)
            string = re.sub(pattern=pattern, string=string, repl=provision)
        return string
    
    
    def change_pointer(self,m,string): 
        group1 = m.group(1).strip();
        group2 = m.group(2).strip();
        pointer_type = -1;
        pointer_of_pointers = -1;

        if group1 == "" or group2 == "" or group1 == "                 " or "=" in group2: 
            return string;

        string_check= re.compile('[@!#$%^&()*<>?\/|[\]}{~:+]')

        if string_check.search(group2) != None:
            if "*" in group2:
                pointer_of_pointers = "Pointer"

            group2_sub = re.sub(r'[^\w]'," ",group2)
            group2_sub = group2_sub.replace(" ","")
            if group2_sub == "" or group2_sub.isdigit():
                return string
            temp =  r'\b'+group2_sub+'\\b'
            pattern = re.compile(temp)

        else:
            group2 = group2.replace(" ","")
            if group2 == "" or group2.isdigit():
                return string
            temp =  r'\b'+group2+'\\b'
            pattern = re.compile(temp)

        if "Pointer" in group2:
            return string;

        elif "Marker" in group2:
            group2_sub = group2.replace("Marker","")
            group2_sub = re.sub(r'[^\w]'," ",group2_sub)
            group2_sub = group2_sub.strip()
            replace = group2_sub + "PointerMarker"
            string = re.sub(pattern=pattern, string=string, repl=replace)
            return string;


        if "(" in group1 and ")" in group1:
            return string;
        else:
            if "," in group1:
                splitted = group1.split(",")
                pointer_type = splitted[len(splitted)-1]
                pointer_type = pointer_type.strip()
                pointer_type_split = pointer_type.split(" ")
            elif "(" in group1:
                splitted = group1.split("(")
                pointer_type = splitted[len(splitted)-1]
                pointer_type = pointer_type.strip()
                pointer_type_split = pointer_type.split(" ")

            elif " " in group1 :
                pointer_type_split = group1.split(" ")

            else:
                pointer_type_split = group1.split(" ")

            if pointer_type_split[0] == "const" and len(pointer_type_split)>1:
                pointer_type = pointer_type_split[1];
            else:
                pointer_type = pointer_type_split[0];

            if  pointer_type != -1:
                pointer_type = pointer_type.replace(" ","")

                if pointer_of_pointers != -1:
                    pointer_type = pointer_of_pointers + pointer_type

                replace = pointer_type + "PointerMarker"
                string = re.sub(pattern=pattern, string=string, repl=replace)
                return string
            return string;


    def find_pointer(self, string):
        regex_index=0     
        while regex_index < len(self.regex_pointer):
            iter = re.finditer(self.regex_pointer[regex_index],string)
            for m in iter:
                string = self.change_pointer(m,string)
            regex_index = regex_index+1
        return string
    
    
    def marker_define(self, string):
        regex_index=0     

        while regex_index < len(self.regex_define):
            m = re.findall(self.regex_define[regex_index],string)
            string = self.change_define(m,string)
            regex_index = regex_index+1
        return string
                
        
    def struct_marker(self, string):
    
        regex_index=0 
        
        while regex_index < len(self.regex_struct):
            r1 = re.findall(self.regex_struct[regex_index], string,flags=re.MULTILINE)
            for elem in r1:
                string_check= re.compile('[@!#$%^&*()<>?=\/|[\]}{~:]')
                if(string_check.search(elem) != None ):

                    if "*" in elem:
                        continue

                    elem = elem.lstrip()
                    new_value = elem.split(" ")

                    if ((")" in new_value[0]) or ("(" in  new_value[0])):
                        continue
                    else:
                        if "=" in new_value[0]:
                            splitedForEqual=  new_value[0].split("=")
                            if splitedForEqual[0] in self.struct_type_array:
                                continue
                            else:    
                                self.struct_type_array.append(splitedForEqual[0])
                        else:
                            if new_value[0] in self.struct_type_array:
                                continue
                            else:
                                self.struct_type_array.append(new_value[0])
                else:
                    elem = elem.lstrip()
                    new_value = elem.split(" ")
                    if new_value[0] in self.struct_type_array:
                        continue
                    else:
                        self.struct_type_array.append(new_value[0])

            regex_index = regex_index + 1
        string = self.struct_type_mark(self.struct_type_array, self.regex_struct_provision[0], string)

        r2 = re.findall("StructMarker{(.*?)}(.*?);", string, flags=re.DOTALL)
        for elem in r2 :
            string_check= re.compile('[@!#$%^&*()<>?=\/|[\]}{~:]')
            elem = elem[len(elem)-1]
            if(string_check.search(elem) != None):
                    elem = re.sub(r'[^\w]'," ",elem)
                    elem = elem.lstrip()
                    new_value = elem.split(" ")
                    self.struct_variable.append(new_value[0])
            else:
                    elem = elem.strip()
                    self.struct_variable.append(elem)
        string = self.struct_variable_mark(self.struct_variable, "StructVariable", string)

        self.struct_variable = []
        r2 = re.findall("StructMarker(.*?);",string,flags=re.MULTILINE)
        for elem in r2 :
            string_check= re.compile('[@!#$%^&*()<>?=\/|[\]}{~:]')
            if(string_check.search(elem) != None):
                    elem = re.sub(r'[^\w]'," ",elem)
                    elem = elem.lstrip()
                    new_value = elem.split(" ")
                    if "{" in new_value[0]:
                        continue
                    else:
                        self.struct_variable.append(new_value[0])
            else:
                    elem = elem.strip()
                    new_value = elem.split(",")
                    if len(new_value)>1:
                        for variable in new_value:
                            variable=variable.strip()
                            self.struct_variable.append(variable)
                    else:
                        self.struct_variable.append(elem)

        string = self.struct_variable_mark(self.struct_variable, "StructVariable", string)
        return string
 
    
        

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class Degisken:
    def __init__(self, start_index, end_index, variable_name, variable_key):
        self.start_index = start_index
        self.end_index = end_index
        self.variable_name = variable_name
        self.variable_key = variable_key


In [ ]:

import pandas as pd
datas = pd.read_csv('drive/MyDrive/kod-bitirme/sard_and_vdisc_vdisc_train_final.csv')


In [ ]:
datas[datas['DataType']=='SARD']

,code,CWE-119,CWE-120,CWE-469,CWE-476,CWE-OTHERS,Label,DataType
1019471,"\n \n \n #include ""std_testcase.h""\n \n #inclu...",False,False,False,False,True,1,SARD
1019472,"\n \n \n #include ""std_testcase.h""\n \n #inclu...",False,False,False,False,True,1,SARD
1019473,"\n \n \n #include ""std_testcase.h""\n \n #inclu...",False,False,False,False,True,1,SARD
1019474,"\n \n \n #include ""std_testcase.h""\n \n #inclu...",False,False,False,False,True,1,SARD
1019475,"\n \n \n #include ""std_testcase.h""\n \n #inclu...",False,False,False,False,True,1,SARD
...,...,...,...,...,...,...,...,...
1067137,"\n \n \n #include ""std_testcase.h""\n \n #defin...",False,False,False,False,True,1,SARD
1067138,"\n \n \n #include ""std_testcase.h""\n \n #defin...",False,False,False,False,True,1,SARD
1067139,"\n \n \n #include ""std_testcase.h""\n \n #defin...",False,False,False,False,True,1,SARD
1067140,"\n \n \n #include ""std_testcase.h""\n \n #defin...",False,False,False,False,True,1,SARD


In [ ]:
import signal
import time
class TimeoutException(Exception): 
    pass

def timeout_handler(signum, frame):  
    raise TimeoutException

signal.signal(signal.SIGALRM, timeout_handler)
counter = 0
error_counter = 0
marked_texts = []

marked_df = pd.DataFrame(columns = ['code','CWE-119','CWE-120','CWE-469','CWE-476','CWE-OTHERS','Label','DataType'])
notmarked_df = pd.DataFrame(columns = ['code','CWE-119','CWE-120','CWE-469','CWE-476','CWE-OTHERS','Label','DataType'])


for index,data in datas.iterrows():
    signal.alarm(8)
    try:
        train_data = preProcess()
        if data['DataType'] == 'VDISC':
            start_index = data['code'].find('(')
            function_name = data['code'][0:start_index]
            temp_text = ' void ' + data['code']
            marked_text = train_data.start(temp_text)
            temp_text = marked_text.replace('void VoidFunctionMarker', str(function_name),1)
            notmarked_df = notmarked_df.append({'code':data['code'], 'CWE-119':data['CWE-119'], 'CWE-120' : data['CWE-120'], 'CWE-469': data['CWE-469'], 'CWE-476':data['CWE-476'], 'CWE-OTHERS': data['CWE-OTHERS'],'Label':data['Label'],'DataType':data['DataType']}, ignore_index=True)
            marked_df = marked_df.append({'code':temp_text, 'CWE-119':data['CWE-119'], 'CWE-120' : data['CWE-120'], 'CWE-469': data['CWE-469'], 'CWE-476':data['CWE-476'], 'CWE-OTHERS': data['CWE-OTHERS'],'Label':data['Label'],'DataType':data['DataType']}, ignore_index=True)

        elif data['DataType'] == 'SARD': 
            temp_text = data['code']
            marked_text = train_data.start(temp_text)
            notmarked_df = notmarked_df.append({'code':data['code'], 'CWE-119':data['CWE-119'], 'CWE-120' : data['CWE-120'], 'CWE-469': data['CWE-469'], 'CWE-476':data['CWE-476'], 'CWE-OTHERS': data['CWE-OTHERS'],'Label':data['Label'],'DataType':data['DataType']}, ignore_index=True)
            marked_df = marked_df.append({'code':marked_text, 'CWE-119':data['CWE-119'], 'CWE-120' : data['CWE-120'], 'CWE-469': data['CWE-469'], 'CWE-476':data['CWE-476'], 'CWE-OTHERS': data['CWE-OTHERS'],'Label':data['Label'],'DataType':data['DataType']}, ignore_index=True)      
        counter = counter + 1

    except TimeoutException:
        print("time out exception")
        continue # continue the for loop if function A takes more than 5 second

    except Exception as e:
      error_counter = error_counter + 1
      continue
    
    else:
        signal.alarm(0)

Görüntülenen çıkış son 5000 satıra kısaltıldı.
1062505
1062506
1062507
1062508
1062509
1062510
1062511
time out exception
1062513
1062514
1062515
1062516
1062517
1062518
1062519
1062520
1062521
1062522
1062523
1062524
1062525
1062526
1062527
1062528
1062529
1062530
1062531
1062532
1062533
1062534
1062535
1062536
1062537
1062538
1062539
1062540
1062541
1062542
1062543
1062544
1062545
time out exception
1062547
1062548
1062549
1062550
1062551
1062552
1062553
1062554
1062555
1062556
1062557
1062558
1062559
1062560
1062561
1062562
1062563
1062564
1062565
1062566
1062567
time out exception
1062569
1062570
1062571
1062572
1062573
1062574
1062575
1062576
1062577
1062578
1062579
1062580
1062581
1062582
1062583
1062584
1062585
1062586
1062587
1062588
1062589
1062590
1062591
1062592
1062593
1062594
1062595
1062596
1062597
1062598
1062599
1062600
1062601
time out exception
1062603
1062604
1062605
1062606
1062607
1062608
1062609
1062610
1062611
1062612
1062613
1062614
1062615
1062616
1062617
10626

In [ ]:
marked_df[marked_df['DataType'] == 'SARD']

,code,CWE-119,CWE-120,CWE-469,CWE-476,CWE-OTHERS,Label,DataType
19405,"\n \n \n #include ""IncludeMarker""\n \n #includ...",False,False,False,False,True,1,SARD
19406,"\n \n \n #include ""IncludeMarker""\n \n #includ...",False,False,False,False,True,1,SARD
19407,"\n \n \n #include ""IncludeMarker""\n \n #includ...",False,False,False,False,True,1,SARD
19408,"\n \n \n #include ""IncludeMarker""\n \n #includ...",False,False,False,False,True,1,SARD
19409,"\n \n \n #include ""IncludeMarker""\n \n #includ...",False,False,False,False,True,1,SARD
...,...,...,...,...,...,...,...,...
66227,"\n \n \n #include ""IncludeMarker""\n \n #define...",False,False,False,False,True,1,SARD
66228,"\n \n \n #include ""IncludeMarker""\n \n #define...",False,False,False,False,True,1,SARD
66229,"\n \n \n #include ""IncludeMarker""\n \n #define...",False,False,False,False,True,1,SARD
66230,"\n \n \n #include ""IncludeMarker""\n \n #define...",False,False,False,False,True,1,SARD


In [ ]:
notmarked_df

,code,CWE-119,CWE-120,CWE-469,CWE-476,CWE-OTHERS,Label,DataType
0,z80_reset(void *param)\r\n{\r\n\tZ80_DaisyChai...,False,False,False,False,False,0,VDISC
1,gfs_l_gfork_free_write_cb(\n globus_xio_han...,False,False,False,False,False,0,VDISC
2,__indirect_glIndexubv(const GLubyte * c)\n{\n ...,False,True,False,False,False,1,VDISC
3,"ensCacheStore(EnsPCache cache, void *key, void...",False,False,False,False,False,0,VDISC
4,g_putch(int in_ch)\n{\n char ch = (char)in_...,False,False,False,False,False,0,VDISC
...,...,...,...,...,...,...,...,...
66228,"\n \n \n #include ""std_testcase.h""\n \n #defin...",False,False,False,False,True,1,SARD
66229,"\n \n \n #include ""std_testcase.h""\n \n #defin...",False,False,False,False,True,1,SARD
66230,"\n \n \n #include ""std_testcase.h""\n \n #defin...",False,False,False,False,True,1,SARD
66231,"\n \n \n #include ""std_testcase.h""\n \n #defin...",False,False,False,False,True,1,SARD


201
66232


In [ ]:
marked_df.to_csv('drive/MyDrive/kod-bitirme/datasets/binary_classification/marked_sard_and_vdisc_vdisc_train_final_1000_son.csv', index=False)
notmarked_df.to_csv('drive/MyDrive/kod-bitirme/datasets/binary_classification/notmarked_sard_and_vdisc_vdisc_train_final_1000_son.csv', index=False)